In [1]:
### Notebook to test decoding of HK
import pandas as pd
import numpy as np
from pathlib import Path
from bitstruct import unpack_from as upf
from pathlib import Path


#Proc_DIR = Path(input("Type the path to the folder where the PROC folder is stored: "))
Proc_DIR = Path('C:/Users/ucasbwh/OneDrive - University College London/PanCam Documents/Rover Level Testing/Data/190809 - PAN_FIT_01/PROC')
## Search for PanCam TM Files
FILT_DIR = "*RAW_TM.pickle"
PikFile = sorted(Proc_DIR.rglob(FILT_DIR))

RTM = pd.read_pickle(PikFile[0])

In [2]:
display(RTM)

,RAW,DT
0,1507000003FA7B9000004D000000000000000000000000...,2019-08-09 13:24:33.982
1,1507000003FB7B9000004D000000000000000000000000...,2019-08-09 13:24:34.981
2,1507000003FC7B9000004D000000000000000000000000...,2019-08-09 13:24:35.979
3,1507000003FD7B9000004D000000000000000000000000...,2019-08-09 13:24:36.977
4,1507000003FE7B9000004D000000000000000000000000...,2019-08-09 13:24:37.992
...,...,...
465,3503000005CB7B9000003D0007090484073B06B806B806...,2019-08-09 13:32:18.858
466,3503000005CC7B9000003D0007090484073B06B806B806...,2019-08-09 13:32:19.857
467,3503000005CD7B9000003D0007090484073B06B806B806...,2019-08-09 13:32:20.855
468,1507000005CE7B9000004D0007090484073B06B806B806...,2019-08-09 13:32:21.853


In [3]:
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))
Bin

0      [21, 7, 0, 0, 3, 250, 123, 144, 0, 0, 77, 0, 0...
1      [21, 7, 0, 0, 3, 251, 123, 144, 0, 0, 77, 0, 0...
2      [21, 7, 0, 0, 3, 252, 123, 144, 0, 0, 77, 0, 0...
3      [21, 7, 0, 0, 3, 253, 123, 144, 0, 0, 77, 0, 0...
4      [21, 7, 0, 0, 3, 254, 123, 144, 0, 0, 77, 0, 0...
                             ...                        
465    [53, 3, 0, 0, 5, 203, 123, 144, 0, 0, 61, 0, 7...
466    [53, 3, 0, 0, 5, 204, 123, 144, 0, 0, 61, 0, 7...
467    [53, 3, 0, 0, 5, 205, 123, 144, 0, 0, 61, 0, 7...
468    [21, 7, 0, 0, 5, 206, 123, 144, 0, 0, 77, 0, 7...
469    [21, 7, 0, 0, 5, 207, 123, 144, 0, 0, 77, 0, 7...
Name: RAW, Length: 470, dtype: object

In [4]:
from bitstruct import unpack_from as upf
import pandas as pd

def PandUPF(Column, Len, OffBy, OffBi):
    """Extracts the RAW value from a binary pandas data column"""
    Extract = Column.apply(lambda x: upf(Len, x, offset=8*OffBy+OffBi)[0]).astype('Int64')
    return Extract

In [5]:
class decodeRAW_IMGHDR_Error(Exception):
    """error for unexpected things"""
    pass

In [11]:
TM = pd.DataFrame()
TM['DT'] = RTM['DT']

# Byte 0-10 TM Block Header
TM['Block_Type']     = PandUPF(Bin, 'u1', 0, 0)
TM['TM_Criticality'] = PandUPF(Bin, 'u2', 0, 1)
TM['MMS_Dest']       = PandUPF(Bin, 'u1', 0, 3)
TM['Instr_ID']       = PandUPF(Bin, 'u4', 0, 4)
TM['TM_Type_ID']     = PandUPF(Bin, 'u6', 0, 8)
TM['Seq_Flag']       = PandUPF(Bin, 'u2', 0, 14)
TM['Pkt_CUC']        = PandUPF(Bin, 'u48', 0, 16)
TM['Data_Len']       = PandUPF(Bin, 'u24', 0, 64)
#Byte 11
if True in (PandUPF(Bin, 'u8', 11, 0) != 0).unique():
    raise decodeRAW_HK_Error("TM Byte 11 not 0")

#Byte 12-17 Voltages
TM['Volt_Ref'] = PandUPF(Bin, 'u16', 12, 0)
TM['Volt_6V0'] = PandUPF(Bin, 'u16', 14, 0)
TM['Volt_1V5'] = PandUPF(Bin, 'u16', 16, 0)

#Byte 18-31 Temperatures
TM['Temp_LFW']  = PandUPF(Bin, 'u16', 18, 0)
TM['Temp_RFW']  = PandUPF(Bin, 'u16', 20, 0)
TM['Temp_HRC']  = PandUPF(Bin, 'u16', 22, 0)
TM['Temp_LWAC'] = PandUPF(Bin, 'u16', 24, 0)
TM['Temp_RWAC'] = PandUPF(Bin, 'u16', 26, 0)

#Byte 32-33 Error Codes
TM['ERR_1'] = PandUPF(Bin, 'u16', 32, 0)
TM['ERR_2'] = PandUPF(Bin, 'u16', 34, 0)
TM['ERR_3'] = PandUPF(Bin, 'u16', 36, 0)

#Byte 38-39 PIU Htr Status
TM['Stat_Temp_Fl'] = PandUPF(Bin, 'u1' , 38, 0)
TM['Stat_Temp_Mo'] = PandUPF(Bin, 'u1' , 38, 1)
TM['Stat_Temp_He'] = PandUPF(Bin, 'u2' , 38, 2)
TM['Stat_Temp_Se'] = PandUPF(Bin, 'u12', 38, 4)

#Byte 40-41 PIU FW Status
if True in (PandUPF(Bin, 'u1', 40, 0) != 0).unique():
    raise decodeRAW_HK_Error("TM Byte 40 bit 0 not 0")
TM['Stat_FWL_Op']  = PandUPF(Bin, 'u1' , 40, 1)
TM['Stat_FWL_Ho']  = PandUPF(Bin, 'u1' , 40, 2)
TM['Stat_FWL_Id']  = PandUPF(Bin, 'u1' , 40, 3)
TM['Stat_FWL_Po']  = PandUPF(Bin, 'u4' , 40, 4)
if True in (PandUPF(Bin, 'u1', 41, 0) !=0).unique():
    raise decodeRAW_HK_Error("TM Byte 41 bit 0 not 0")
TM['Stat_FWR_Op']  = PandUPF(Bin, 'u1' , 41, 1)
TM['Stat_FWR_Ho']  = PandUPF(Bin, 'u1' , 41, 2)
TM['Stat_FWR_Id']  = PandUPF(Bin, 'u1' , 41, 3)
TM['Stat_FWR_Po']  = PandUPF(Bin, 'u4' , 41, 4)

#Byte 42-43 PIU Cam Status
TM['Stat_PIU_En']  = PandUPF(Bin, 'u8', 42, 0)
TM['Stat_PIU_Pw']  = PandUPF(Bin, 'u8', 43, 0)

#Byte 44-45 Camera Response

#Byte 64-71 Filter Wheel
TM['FWL_ABS'] = PandUPF(Bin, 'u16', 64, 0)
TM['FWR_ABS'] = PandUPF(Bin, 'u16', 66, 0)
TM['FWL_REL'] = PandUPF(Bin, 'u16', 68, 0)
TM['FWR_REL'] = PandUPF(Bin, 'u16', 70, 0)

## Non-Essential Only HK
NEBin = Bin[TM['TM_Type_ID']==1]
if not NEBin.empty:
    #Byte 72-77 Image ID
    TM['IMG_SOL'] = PandUPF(NEBin, 'u12', 72, 0)
    TM['IMG_Task_ID'] = PandUPF(NEBin, 'u7', 73, 4)
    TM['IMG_Task_Run_No'] = PandUPF(NEBin, 'u7', 74, 3)
    TM['IMG_Cam'] = PandUPF(NEBin, 'u2', 75, 2)
    TM['IMG_FW'] = PandUPF(NEBin, 'u4', 75, 4)
    TM['IMG_No'] = PandUPF(NEBin, 'u8', 76, 0)
    if True in (PandUPF(NEBin, 'u1', 77, 0) !=0).unique():
        raise decodeRAW_HK_Error("TM Byte 77 not 0")
    
    #Byte 78-79 PIU Version
    TM['PIU_Ver'] = PandUPF(NEBin, 'u16', 78, 0)

    #Byte 80-87 FW Config
    TM['FWL_RTi'] = PandUPF(NEBin, 'u8',  80, 0)
    TM['FWL_Spe'] = PandUPF(NEBin, 'u4',  81, 0)
    TM['FWR_Spe'] = PandUPF(NEBin, 'u4',  81, 4)
    TM['FWL_Cur'] = PandUPF(NEBin, 'u16', 82, 0)
    TM['FWR_Cur'] = PandUPF(NEBin, 'u16', 84, 0)
    TM['FWR_RTi'] = PandUPF(NEBin, 'u8',  86, 0)
    TM['FWL_StL'] = PandUPF(NEBin, 'u4',  87, 0)
    TM['FWR_StR'] = PandUPF(NEBin, 'u4',  87, 4)
    del NEBin

##Byte 44-63 WAC Camera Responses
WACBin = Bin[TM['Stat_PIU_Pw'].between(1,2)]
HRCBin = Bin[TM['Stat_PIU_Pw'] == 3]
if not WACBin.empty:
    TM['WAC_CID'] = PandUPF(WACBin, 'u2', 44, 0)
    if False in PandUPF(WACBin, 'u1', 44, 2):
        raise decodeRAW_HK_Error("TM Byte 44 bit 2 not 0 for WAC")
    TM['WAC_WID'] = PandUPF(WACBin, 'u3', 44, 5)
    TM['WAC_WTS'] = PandUPF(WACBin, 'u48', 51, 0)
    TM['WAC_SUM'] = PandUPF(WACBin, 'u8', 59, 0)
    TM.WAC_SUM[TM['WAC_CID'] == 2] = np.NaN #Set WAC DT Checksums to 0 as don't exist

    #WAC IA
    WIA = WACBin[TM['WAC_CID'] == 0]
    if not WIA.empty:
        TM['WAC_IAS'] = PandUPF(WIA, 'u2', 44, 3)
        if True in (PandUPF(WIA, 'u48', 45, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 45-50 not 0 for WAC IA")
        if True in (PandUPF(WIA, 'u16', 57, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 57-58 not 0 for WAC IA")
        if True in (PandUPF(WIA, 'u32', 60, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 60-63 not 0 for WAC IA")
        del WIA

    #WAC HK
    WHK = WACBin[TM['WAC_CID'] == 1]
    if not WHK.empty:
        TM['WAC_HK_MCK'] = PandUPF(WHK, 'u2' , 44, 3)
        TM['WAC_HK_TAT'] = PandUPF(WHK, 'u48', 45, 0)
        TM['WAC_HK_LTP'] = PandUPF(WHK, 'u12', 57, 0)
        TM['WAC_HK_INH'] = PandUPF(WHK, 'u1' , 58, 4)
        TM['WAC_HK_IAO'] = PandUPF(WHK, 'u1' , 58, 5)
        TM['WAC_HK_TAO'] = PandUPF(WHK, 'u1' , 58, 6)
        TM['WAC_HK_MCO'] = PandUPF(WHK, 'u1' , 58, 7)
        if True in (PandUPF(WHK, 'u32', 60, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 60-63 not 0 for WAC HK")
        del WHK

    #WAC DT
    WDT = WACBin[TM['WAC_CID'] == 2]
    if not WDT.empty:
        TM['WAC_DT_BIN'] = PandUPF(WDT, 'u2' , 44, 3)
        TM['WAC_DT_ITS'] = PandUPF(WDT, 'u48', 45, 0)
        TM['WAC_DT_INT'] = PandUPF(WDT, 'u20', 57, 0)
        TM['WAC_DT_STP'] = PandUPF(WDT, 'u12', 59, 4)
        TM['WAC_DT_INH'] = PandUPF(WDT, 'u1',  61, 0)
        TM['WAC_DT_AE']  = PandUPF(WDT, 'u1',  61, 1)
        TM['WAC_DT_PAD'] = PandUPF(WDT, 'u1',  61, 2)
        TM['WAC_DT_GAS'] = PandUPF(WDT, 'u2',  61, 3)
        TM['WAC_DT_DD']  = PandUPF(WDT, 'u1',  61, 5)
        TM['WAC_DT_AES'] = PandUPF(WDT, 'u1',  61, 6)
        if True in (PandUPF(WDT, 'u1', 61, 7) !=0).unique():
            raise decodeRAW_HK_Error("TM Byte 71 bit 7 not 0 for WAC DT")
        TM['WAC_DT_CRC'] = PandUPF(WDT, 'u16', 62, 0)
        del WDT

    #WAC NAK
    WNK = WACBin[TM['WAC_CID'] == 3]
    if not WNK.empty:
        TM['WAC_NK_ERR'] = PandUPD(WNK, 'u8', 45, 0)
        if True in (PandUPD(WNK, 'u40', 46, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 46-50 not 0 for WAC NAK")
        if True in (PandUPD(WNK, 'u16', 57, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 57-58 not 0 for WAC NAK")
        if True in (PandUPF(WNK, 'u32', 60, 0) !=0).unique():
            raise decodeRAW_HK_Error("TM Bytes 60-63 not 0 for WAC NAK")
        del WNK
        
print("Done 2")

Done 2


In [28]:
## Select all Cam Response when a camera is powered off
with pd.option_context('display.max_rows', -1, 'display.max_columns', 20):
    display(TM[['WAC_CID','WAC_SUM']])

,WAC_CID,WAC_SUM
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
